# The adverse health effects of air pollution - are we making any progress?

<img src=files/air.jpg width="900">

**Credit:**  [Flickr/E4C](https://www.flickr.com/people/44221799@N08/) 

In [1]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings

warnings.filterwarnings("ignore")  # Suppress all warnings

## Introduction

**Business Context.** Air pollution is a very serious issue that the global population is currently dealing with. The abundance of air pollutants is not only contributing to global warming, but it is also causing problematic health issues to the population. There have been numerous efforts to protect and improve air quality across most nations. However, it seems that we are making very little progress. One of the main causes of this is the fact that the majority of air pollutants are derived from the burning of fossil fuels such as *coal*. Big industries and several other economical and political factors have slowed the progress towards the use of renewable energy by promoting the use of fossil fuels. Nevertheless, if we educate the general population and create awareness of this issue, we will be able to overcome this problem in the future.      

For this case, you have been hired as a data science consultant for an important environmental organization. In order to promote awareness of environmental and greenhouse gas issues, your client is interested in a study of **plausible impacts of air contamination on the health of the global population**. They have gathered some *raw* data provided by the [World Health Organization](https://www.who.int/), [The Institute for Health Metrics and Evaluation](http://www.healthdata.org/) and the [World Bank Group](https://www.worldbank.org/). Your task is to conduct data analysis, search for potential information, and create visualizations that the client can use for their campaigns and grant applications. 

**Analytical Context.** You are given a folder, named ```files``` with *raw* data. This data contains quite a large number of variables and it is in a fairly disorganized state. In addition, one of the datasets contains very poor documentation, segmented into several datasets. Your objective will be to:

1. Extract and clean the relevant data. You will have to manipulate several datasets to obtain useful information for the case. 
2. Conduct Exploratory Data Analysis. You will have to create meaningful plots, formulate meaningful hypotheses and study the relationship between various indicators related to air pollution.

Additionally, the client has some broad questions they would like to answer:
1. Are we making any progress in reducing the amount of emitted pollutants across the globe?
2. Which are the critical regions where we should start environmental campaigns?
3. Are we making any progress in the prevention of deaths related to air pollution?
4. Which demographic characteristics seem to correlate with the number of health-related issues derived from air pollution? 

## Extracting and cleaning relevant data

Let's take a look at the data provided by the client in the ```files``` folder. There, we see another folder  named ```WDI_csv``` with several CSV files corresponding to the World Bank's primary [World Development Indicators](https://datacatalog.worldbank.org/dataset/world-development-indicators). The client stated that this data may contain some useful information relevant to our study, but they have not told us anything aside from that. Thus, we are on our own in finding and extracting the relevant data for our study. This we will do next. 

Let's take a peek at the file ```WDIData.csv```:

In [2]:
WDI_data = pd.read_csv("./files/WDI_csv/WDIData.csv")
print(WDI_data.columns)
print(WDI_data.info())
WDI_data.head()

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', 'Unnamed: 64'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377256 entries, 0 to 377255
Data columns (total 65 columns):
Country Name      377256 non-null object
Country Code      377256 non-null object
Indicator Name    377256 non-null object
Indicator Code    377256 non-null object
1960              37395 non-null float64
1961              41211 non-null float64
1

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.783289,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN
3,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.428272,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN,NaN
4,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,73.942103,75.244104,77.162305,75.538976,78.741152,79.665635,80.749293,NaN,NaN,NaN


The data seems to have a large number of indicators dating from 1960. There are also columns containing country names and codes. Notice that the first couple of rows say ```Arab World```, which may indicate that the data contains broad regional data as well. We notice also that there are at least 100,000 entries with ```NaN``` values for each year column.

Since we are interested in environmental indicators, we must get rid of any rows not relevant to our study. However, the number of indicators seems to be quite large and a manual inspection seems impossible. Let's load the file ```WDISeries.csv``` which seems to contain more information about the indicators:

In [3]:
WDI_ids = pd.read_csv("./files/WDI_csv/WDISeries.csv")
print(WDI_ids.columns)
WDI_ids.head()

Index(['Series Code', 'Topic', 'Indicator Name', 'Short definition',
       'Long definition', 'Unit of measure', 'Periodicity', 'Base Period',
       'Other notes', 'Aggregation method', 'Limitations and exceptions',
       'Notes from original source', 'General comments', 'Source',
       'Statistical concept and methodology', 'Development relevance',
       'Related source links', 'Other web links', 'Related indicators',
       'License Type', 'Unnamed: 20'],
      dtype='object')


,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,...,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
0,AG.AGR.TRAC.NO,Environment: Agricultural production,"Agricultural machinery, tractors",NaN,Agricultural machinery refers to the number of...,NaN,Annual,NaN,NaN,Sum,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",A tractor provides the power and traction to m...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN
1,AG.CON.FERT.PT.ZS,Environment: Agricultural production,Fertilizer consumption (% of fertilizer produc...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,NaN,Weighted average,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0,NaN
2,AG.CON.FERT.ZS,Environment: Agricultural production,Fertilizer consumption (kilograms per hectare ...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,NaN,Weighted average,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0,NaN
3,AG.LND.AGRI.K2,Environment: Land use,Agricultural land (sq. km),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,NaN,Sum,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agricultural land constitutes only a part of a...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN
4,AG.LND.AGRI.ZS,Environment: Land use,Agricultural land (% of land area),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,NaN,Weighted average,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agriculture is still a major sector in many ec...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN


Bingo! The ```WDI_ids``` DataFrame contains a column named ```Topic```. Moreover, it seems that *Environment* is listed as a key topic in the column.

### Exercise 1:

Extract all the rows that have the topic key *Environment* in ```WDI_ids```. Add to the resulting DataFrame a new column named ```Subtopic``` which contains the corresponding subtopic of the indicator. For example, the subtopic of ```Environment: Agricultural production``` is ```Agricultural production```. Which subtopics do you think are of interest to us?

**Hint:** Remember that you can apply string methods to Series using the ```str()``` method of ```pandas```. 

**Answer.**

-------

We may find several of these subtopics relevant to our study, but probably the most interesting for us now is the ```Emissions``` subtopic.

### Exercise 2:

Use the results of Exercise 1 to create a new DataFrame with the history of all emissions indicators for countries and major regions. Call this new DataFrame ```Emissions_df```. How many emissions indicators are in the study?

**Answer.**

-------

### Exercise 3:

The DataFrame ```Emissions_df``` has one column per year of observation. Data in this form is usually referred to as data in *wide format*, as the number of columns is high. However, it might be easier to query and filter the data if we had a single column containing the year in which each indicator was calculated. This way, *each observation will be represented by a single row*. Use the ```pandas``` function [```melt()```](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) to reshape the ```Emissions_df``` data into *long format*. The resulting DataFrame should contain a pair of new columns named ```Year``` and ```Indicator Value```:

**Answer.**

-------

### Exercise 4:

The column ```Indicator Value``` of the new ```Emissions_df``` contains a bunch of ```NaN``` values. Additionally, the ```Year``` column contains an ```Unnamed: 64``` value. What procedure should we follow to clean these missing values in our DataFrame? Proceed with your suggested cleaning process.

**Answer.**

-------

### Exercise 5:

Split the ```Emissions_df``` into two DataFrames, one containing only countries and the other containing only regions. Name these ```Emissions_C_df``` and ```Emissions_R_df``` respectively.

**Hint:** You may want to inspect the file ```WDICountry.csv``` for this task. Region country codes may be found by looking at ```null``` values of the ```Region``` column in ```WDICountry```.

**Answer.**

-------

## Finalizing the cleaning for our study

Our data has improved a lot by now. However, since the number of indicators is still quite large, let us focus our study on the following indicators for now:

- **Total greenhouse gas emissions (kt of CO2 equivalent), EN.ATM.GHGT.KT.CE**: The total of greenhouse emissions includes CO2, Methane, Nitrous oxide, among other pollutant gases. Measured in kilotons.


- **CO2 emissions (kt), EN.ATM.CO2E.KT**: Carbon dioxide emissions are those stemming from the burning of fossil fuels and the manufacture of cement. They include carbon dioxide produced during consumption of solid, liquid, and gas fuels and gas flaring. 	


- **Methane emissions (kt of CO2 equivalent), EN.ATM.METH.KT.CE**: Methane emissions are those stemming from human activities such as agriculture and from industrial methane production.


- **Nitrous oxide emissions (kt of CO2 equivalent), EN.ATM.NOXE.KT.CE**: Nitrous oxide emissions are emissions from agricultural biomass burning, industrial activities, and livestock management.


- **Other greenhouse gas emissions, HFC, PFC and SF6 (kt of CO2 equivalent), EN.ATM.GHGO.KT.CE**: Other pollutant gases.


- **PM2.5 air pollution, mean annual exposure (micrograms per cubic meter), EN.ATM.PM25.MC.M3**: Population-weighted exposure to ambient PM2.5 pollution is defined as the average level of exposure of a nation's population to concentrations of suspended particles measuring less than 2.5 microns in aerodynamic diameter, which are capable of penetrating deep into the respiratory tract and causing severe health damage. Exposure is calculated by weighting mean annual concentrations of PM2.5 by population in both urban and rural areas.


- **PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total), EN.ATM.PM25.MC.ZS**: Percent of population exposed to ambient concentrations of PM2.5 that exceed the World Health Organization (WHO) guideline value.

### Exercise 6:

For each of the emissions DataFrames, extract the rows corresponding to the above indicators of interest. Replace the long names of the indicators by the short names ```Total```, ```CO2```, ```CH4```, ```N2O```, ```Other```, ```PM2.5```, and ```PM2.5_WHO```. (This will be helpful later when we need to label plots of our data.) 

**Answer.**

---------

## Where shall the client start environmental campaigns?

Now the DataFrames ```Emissions_C_df``` and ```Emissions_R_df``` seem to be in a good shape. Let's proceed to conduct some exploratory data analysis so that we can make recommendations to our client.

### Exercise 7:

 Let's first calculate some basic information about the main indicators across the globe.

#### 7.1
Compute some basic statistics of the amount of kt of emissions for each of the four main pollutants (```CO2, CH4, N2O, Others```) over the years. Use the ```Emissions_C_df``` data frame. What trends do you see? 

**Answer.**

---------

#### 7.2

What can you say about the *distribution* of emissions around the globe over the years? What information can you extract from the *tails* of these distributions over the years?

**Answer.**

---------

#### 7.3
Compute a plot showing the behavior of each of the four main air pollutants for each of the main global regions in the ```Emissions_R_df``` data frame. The main regions are ```'Latin America & Caribbean', 'South Asia', 'Sub-Saharan Africa', 'Europe & Central Asia', 'Middle East & North Africa', 'East Asia & Pacific'``` and ```'North America'```. What conclusions can you make?

**Answer.**

---------

### Exercise 8:

In Exercise 7 we discovered some interesting features of the distribution of the emissions over the years. Let us explore these features in more detail. 

#### 8.1
Which are the top five countries that have been in the top 10 of ```CO2``` emitters over the years? Have any of these countries made efforts to reduce the amount of CO2 emissions over the last 10 years?

**Answer.**

---------

#### 8.2

Are these five countries carrying out the burden of most of the emissions emitted over the years globally? Can we say that the rest of the world is making some effort to control their polluted gasses emissions over the years?

**Answer.**

---------

## The health impacts of air pollution

### Exercise 9:

One of the main contributions of poor health from air pollution is particulate matter. In particular, very small particles (those with a size less than 2.5 micrometres ($\mu$m)) can enter and affect the respiratory system. The ```PM2.5``` indicator measures the average level of exposure of a nation's population to concentrations of these small particles. The ```PM2.5_WHO``` measures the  percentage of the population who are exposed to ambient concentrations of these particles that exceed some thresholds set by the World Health Organization (WHO). In particular, countries with a higher ```PM2.5_WHO``` indicator are more likely to suffer from bad health conditions. 


#### 9.1

The client would like to know if there is any relationship between the ```PM2.5_WHO``` indicator and the level of income of the general population, as well as how this changes over time. What plot(s) might be helpful to solve the client's question?  What conclusion can you draw from your plot(s) to answer their question?

**Hint:** The DataFrame ```WDI_countries``` contains a column named ```Income Group```. 

**Answer.**

---------

#### 9.2

What do you think are the causes behind the results in Exercise 9.1?

**Answer.**

---------

### Exercise 10:

Finally, our client is interested in investigating the impacts and relationships between **high levels of exposure to particle matter** and **the health of the population**. Coming up with additional data for this task may be infeasible for the client, thus they have asked us to search for relevant health data in the ```WDIdata.csv``` file and work with that. 

#### 10.1

Which indicators present in the file  ```WDISeries.csv``` file might be useful to solve the client's question? Explain.

**Note:** Naming one or two indicators is more than enough for this question. 

**Answer.**

---------

#### 10.2

Use the indicators provided in Exercise 10.1 to give valuable information to the client. 

**Answer.**

---------

#### 10.3

Extend the analysis above to find some countries of interest. These are defined as

* The countries that have a high mortality rate due to household and ambient air pollution, but with low PM2.5 exposure
* The countries that have a low mortality rate due to household and ambient air pollution, but with high PM2.5 exposure

**Answer.**

---------

#### 10.4

Finally, we want to look at the mortality data by income. We expect higher income countries to have lower pollution-related mortality. Find out if this assumption holds. Calculate summary statistics and histograms for each income category and note any trends.

**Answer.**

---------

#### 10.5

At the start, we asked some questions. Based on your analysis, provide a short answer to each of these:

1. Are we making any progress in reducing the amount of emitted pollutants across the globe?
2. Which are the critical regions where we should start environmental campaigns?
3. Are we making any progress in the prevention of deaths related to air pollution?
4. Which demographic characteristics seem to correlate with the number of health-related issues derived from air pollution? 

**Answer.**

---------